# Parameters

In [1]:
pn = None #program names
ai= None #strings - account IDs 
dat= '2019-01-01' #date string 
#dat=None
fi= (1620,13307, 10232, 4761) #strings - Falcon account IDs 
lim= 10000 #limit query

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sqlalchemy
from IPython.display import display, HTML, Latex
from matplotlib import cm
from unidecode import unidecode
import os
import re
import json
from datetime import datetime 
import sys
import itertools
from sklearn import metrics
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from functools import reduce


import plotly
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)


%matplotlib inline

# pandas table display format
pd.set_option('display.float_format', '{:.4f}'.format)


# progress bars for time consuming tasks
from tqdm import tqdm
tqdm.pandas(desc='Pandas progress')

# suppress warnings -- helps when notebook has to be exported as a report
import warnings
warnings.filterwarnings('ignore')

# Change dir
os.chdir('../research-discovery/')
os.getcwd()

# Import tools for queries acquisition
from ds_tools import db_tools
from sqlalchemy import MetaData
from ds_tools.misc_tools import batch_iterator

## Helper Functions

In [3]:
def rem_outliers(arr): #Keep 90 percentile
    listt=[]
    count=0
    mmaxx= np.percentile(arr,90)    
    for x in arr:
        if x<mmaxx and x>0:
            listt.append(x)
        else:
            count+=1
    return np.array(listt),count #returns an array without outliers and the number of outliers removed

def get_account_filter(account_ids):
    if (account_ids):
        return 'and p."accountProgramId" IN {}'.format(account_ids)
    return ''

def get_program_name_filter(program_names):
    if (program_names):
        return 'and p."programName" IN {}'.format(program_names)
    return ''

def get_falcon_account_filter(falc_acc):
    if (falc_acc):
        return 'and p."falconAccountId" IN {}'.format(falc_acc)
    return ''

def get_date_filter(date):
    if (date):
        return """and p."createdAt" > '{}' """.format(date)
    return ''

def subtract_times(nparry,col_end,col_start):
    l=[]
    print('shape before: ', nparry.shape)
    for i in range (len(nparry[:,])): #for every row
        l.append(np_data[i,col_end]-nparry[i,col_start]) 
    times=[]    
    for i in range (len(l)): #convert to hours
        times.append(l[i]/np.timedelta64(1, 'h'))

    times=np.array(times).reshape(len(times),1) #convert to np array of dim (len,1)

    nparry = np.append(nparry,times,1) 
    print ('shape after: ' , nparry.shape)
    return nparry

# Pantheon Queries

In [4]:
query=\
'''select p."id" as project_id, p."accountProgramId",p."createdAt" as project_created_time
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='intake-phase'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='start'
limit {}
'''.format(get_falcon_account_filter(fi),get_program_name_filter(pn),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d0 = pd.read_sql(query, conn)

    
#Grey Area
query=\
'''select p."id" as project_id,e."actionTime" as intake_phase_starts
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='intake-phase'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='start'
limit {}
'''.format(get_falcon_account_filter(fi),get_program_name_filter(pn),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d1 = pd.read_sql(query, conn)


query=\
'''select p."id" as project_id,e."actionTime" as intake_task_starts
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='intake'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='start'
limit {}
'''.format(get_falcon_account_filter(fi),get_program_name_filter(pn),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d2 = pd.read_sql(query, conn)


    
query=\
'''select p."id" as project_id ,e."actionTime" as intake_phase_complete
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='intake-phase'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='complete'
limit {}
'''.format(get_falcon_account_filter(fi),get_program_name_filter(pn),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d3 = pd.read_sql(query, conn)



query=\
'''select p."id" as project_id, e."actionTime" as scoping_phase_starts
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='scoping-phase'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='start'
limit {}
'''.format(get_program_name_filter(pn),get_falcon_account_filter(fi),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d4 = pd.read_sql(query, conn)

    
query=\
'''select p."id" as project_id,min(e."actionTime") as scoping_task_assigned
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='scoping'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='start'
group by p."id"
limit {}
'''.format(get_program_name_filter(pn),get_falcon_account_filter(fi),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d5 = pd.read_sql(query, conn)

    
query=\
'''select p."id" as project_id, e."actionTime" as scoping_phase_ends
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='scoping-phase'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='complete'
limit {}
'''.format(get_program_name_filter(pn),get_falcon_account_filter(fi),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d6 = pd.read_sql(query, conn)


query=\
'''select p."id" as project_id, e."actionTime" as approval_phase_starts
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='approval-phase'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='start'
limit {}
'''.format(get_program_name_filter(pn),get_falcon_account_filter(fi),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d7 = pd.read_sql(query, conn)

    
query=\
'''select p."id" as project_id, e."actionTime" as approval_task_starts
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='approval'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='start'
limit {}
'''.format(get_program_name_filter(pn),get_falcon_account_filter(fi),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d8 = pd.read_sql(query, conn)


query=\
'''
select 
p."id" as project_id, min(q."createdAt") as first_quote
from projects p
inner join quotes q on p."id"=q."projectId"
{}
{}
{}
group by p."id"
limit {}
'''.format(get_date_filter(dat),get_program_name_filter(pn),get_falcon_account_filter(fi),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d9 = pd.read_sql(query, conn)
    
    
#Queries for the Yellow Area

query=\
'''select p."id" as project_id,e."actionTime" as confirmIntakeKit_phase_starts
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='approval-phase'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='complete'
limit {}
'''.format(get_falcon_account_filter(fi),get_program_name_filter(pn),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d10 = pd.read_sql(query, conn)
    

query=\
'''select p."id" as project_id,min(e."actionTime") as confirmIntakeKit_task_assigned
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='confirm-intake-kit'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='assign'
group by p."id"
limit {}
'''.format(get_program_name_filter(pn),get_falcon_account_filter(fi),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d11 = pd.read_sql(query, conn)

    
query=\
'''select p."id" as project_id,e."actionTime" as confirmIntakeKit_phase_ends
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='confirm-intake-kit-phase'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='complete'
limit {}
'''.format(get_program_name_filter(pn),get_falcon_account_filter(fi),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d12 = pd.read_sql(query, conn)
    
    
query=\
'''select p."id" as project_id,min(e."actionTime") as planning_Phase_Begin
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='planning'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='assign'
group by p."id"
limit {}
'''.format(get_program_name_filter(pn),get_falcon_account_filter(fi),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d13 = pd.read_sql(query, conn)

query=\
'''select p."id" as project_id,e."actionTime" as planning_task_start
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='planning'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='start'
limit {}
'''.format(get_program_name_filter(pn),get_falcon_account_filter(fi),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d14 = pd.read_sql(query, conn)    
    
query=\
'''select p."id" as project_id,e."actionTime" as planning_Phase_End
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='planning-phase'
{}
{}
{}
inner join events e
on e."refId"= t."id"
and e."refModel"='task'
and e."action"='complete'
limit {}
'''.format(get_program_name_filter(pn),get_falcon_account_filter(fi),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d15 = pd.read_sql(query, conn)    
    
query=\
'''
select 
p."id" as project_id, min(o."createdAt") as first_Offer_sent
from projects p
inner join tasks t on p."id"=t."projectId"
inner join offers o on t."id"=o."taskId"
{}
{}
{}
group by p."id"
limit {}
'''.format(get_date_filter(dat),get_program_name_filter(pn),get_falcon_account_filter(fi),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d16 = pd.read_sql(query, conn)
    
    
#Queries for efforts

#intake effort
#later (need to calculate: intake ends-intake start)    

#query for scoping
query=\
'''
select p."id" as project_id, sum(t."taskDuration") as scoping_effort
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='scoping'
{}
{}
{}
group by p."id", t."type"
limit {}
'''.format(get_falcon_account_filter(fi),get_program_name_filter(pn),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d17 = pd.read_sql(query, conn)



#query for approval
query=\
'''
select p."id" as project_id, sum(t."taskDuration") as approval_effort
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='approval'
{}
{}
{}
group by p."id", t."type"
limit {}
'''.format(get_falcon_account_filter(fi),get_program_name_filter(pn),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d18 = pd.read_sql(query, conn)

#query for CI kit
query=\
'''
select p."id" as project_id, sum(t."taskDuration") as intake_kit_effort
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='confirm-intake-kit'
{}
{}
{}
group by p."id", t."type"
limit {}
'''.format(get_falcon_account_filter(fi),get_program_name_filter(pn),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d19 = pd.read_sql(query, conn)
    
#query for planning
query=\
'''
select p."id" as project_id, sum(t."taskDuration") as planning_effort
from projects p
inner join tasks t
on p."id"=t."projectId"
and t."type"='planning'
{}
{}
{}
group by p."id", t."type"
limit {}
'''.format(get_falcon_account_filter(fi),get_program_name_filter(pn),get_date_filter(dat),lim)

#connecting to the database 
with db_tools.connect_to_db('pantheon_production').connect() as conn:
    d20 = pd.read_sql(query, conn)

## Merging Queries

In [5]:
#merging all queries
dfs=[]
for i in range (21):
    dfs.append(eval('d{}'.format(i)))
df=reduce(lambda left,right: pd.merge(left,right, on='project_id'),dfs) #rolling computation to sequential pairs of values in a list
#convert to numpy array
np_data=df.values #convert pd dataframe to np array

print(df.columns)
df

Index([u'project_id', u'accountProgramId', u'project_created_time',
       u'intake_phase_starts', u'intake_task_starts', u'intake_phase_complete',
       u'scoping_phase_starts', u'scoping_task_assigned',
       u'scoping_phase_ends', u'approval_phase_starts',
       u'approval_task_starts', u'first_quote',
       u'confirmintakekit_phase_starts', u'confirmintakekit_task_assigned',
       u'confirmintakekit_phase_ends', u'planning_phase_begin',
       u'planning_task_start', u'planning_phase_end', u'first_offer_sent',
       u'scoping_effort', u'approval_effort', u'intake_kit_effort',
       u'planning_effort'],
      dtype='object')


,project_id,accountProgramId,project_created_time,intake_phase_starts,intake_task_starts,intake_phase_complete,scoping_phase_starts,scoping_task_assigned,scoping_phase_ends,approval_phase_starts,...,confirmintakekit_task_assigned,confirmintakekit_phase_ends,planning_phase_begin,planning_task_start,planning_phase_end,first_offer_sent,scoping_effort,approval_effort,intake_kit_effort,planning_effort
0,45563,2403386,2019-07-01 03:32:27.311000+00:00,2019-07-01 03:32:50.629000+00:00,2019-07-05 06:10:37.021000+00:00,2019-07-05 06:14:34.061000+00:00,2019-07-05 06:14:51.062000+00:00,2019-07-05 06:14:51.115000+00:00,2019-07-05 06:15:27.075000+00:00,2019-07-05 06:15:30.488000+00:00,...,2019-07-05 06:16:23.545000+00:00,2019-07-05 06:16:47.122000+00:00,2019-07-05 06:16:52.451000+00:00,2019-07-05 06:16:52.955000+00:00,2019-07-05 06:16:58.903000+00:00,2019-07-05 06:17:44.224000+00:00,60.0000,180.0000,60.0000,60.0000
1,45563,2403386,2019-07-01 03:32:27.311000+00:00,2019-07-01 03:32:50.629000+00:00,2019-07-05 06:10:37.021000+00:00,2019-07-05 06:14:34.061000+00:00,2019-07-05 06:14:51.062000+00:00,2019-07-05 06:14:51.115000+00:00,2019-07-05 06:15:27.075000+00:00,2019-07-05 06:15:30.488000+00:00,...,2019-07-05 06:16:23.545000+00:00,2019-07-05 06:16:47.122000+00:00,2019-07-05 06:16:52.451000+00:00,2019-07-05 06:16:52.955000+00:00,2019-07-05 06:16:58.903000+00:00,2019-07-05 06:17:44.224000+00:00,60.0000,180.0000,60.0000,60.0000
2,45563,2403386,2019-07-01 03:32:27.311000+00:00,2019-07-01 03:32:50.714000+00:00,2019-07-05 06:10:37.021000+00:00,2019-07-05 06:14:34.061000+00:00,2019-07-05 06:14:51.062000+00:00,2019-07-05 06:14:51.115000+00:00,2019-07-05 06:15:27.075000+00:00,2019-07-05 06:15:30.488000+00:00,...,2019-07-05 06:16:23.545000+00:00,2019-07-05 06:16:47.122000+00:00,2019-07-05 06:16:52.451000+00:00,2019-07-05 06:16:52.955000+00:00,2019-07-05 06:16:58.903000+00:00,2019-07-05 06:17:44.224000+00:00,60.0000,180.0000,60.0000,60.0000
3,45563,2403386,2019-07-01 03:32:27.311000+00:00,2019-07-01 03:32:50.714000+00:00,2019-07-05 06:10:37.021000+00:00,2019-07-05 06:14:34.061000+00:00,2019-07-05 06:14:51.062000+00:00,2019-07-05 06:14:51.115000+00:00,2019-07-05 06:15:27.075000+00:00,2019-07-05 06:15:30.488000+00:00,...,2019-07-05 06:16:23.545000+00:00,2019-07-05 06:16:47.122000+00:00,2019-07-05 06:16:52.451000+00:00,2019-07-05 06:16:52.955000+00:00,2019-07-05 06:16:58.903000+00:00,2019-07-05 06:17:44.224000+00:00,60.0000,180.0000,60.0000,60.0000
4,45563,2403386,2019-07-01 03:32:27.311000+00:00,2019-07-01 03:32:50.629000+00:00,2019-07-05 06:10:37.021000+00:00,2019-07-05 06:14:34.061000+00:00,2019-07-05 06:14:51.062000+00:00,2019-07-05 06:14:51.115000+00:00,2019-07-05 06:15:27.075000+00:00,2019-07-05 06:15:30.488000+00:00,...,2019-07-05 06:16:23.545000+00:00,2019-07-05 06:16:47.122000+00:00,2019-07-05 06:16:52.451000+00:00,2019-07-05 06:16:52.955000+00:00,2019-07-05 06:16:58.903000+00:00,2019-07-05 06:17:44.224000+00:00,60.0000,180.0000,60.0000,60.0000
5,45563,2403386,2019-07-01 03:32:27.311000+00:00,2019-07-01 03:32:50.629000+00:00,2019-07-05 06:10:37.021000+00:00,2019-07-05 06:14:34.061000+00:00,2019-07-05 06:14:51.062000+00:00,2019-07-05 06:14:51.115000+00:00,2019-07-05 06:15:27.075000+00:00,2019-07-05 06:15:30.488000+00:00,...,2019-07-05 06:16:23.545000+00:00,2019-07-05 06:16:47.122000+00:00,2019-07-05 06:16:52.451000+00:00,2019-07-05 06:16:52.955000+00:00,2019-07-05 06:16:58.903000+00:00,2019-07-05 06:17:44.224000+00:00,60.0000,180.0000,60.0000,60.0000
6,45563,2403386,2019-07-01 03:32:27.311000+00:00,2019-07-01 03:32:50.714000+00:00,2019-07-05 06:10:37.021000+00:00,2019-07-05 06:14:34.061000+00:00,2019-07-05 06:14:51.062000+00:00,2019-07-05 06:14:51.115000+00:00,2019-07-05 06:15:27.075000+00:00,2019-07-05 06:15:30.488000+00:00,...,2019-07-05 06:16:23.545000+00:00,2019-07-05 06:16:47.122000+00:00,2019-07-05 06:16:52.451000+00:00,2019-07-05 06:16:52.955000+00:00,2019-07-05 06:16:58.903000+00:00,2019-07-05 06:17:44.224000+00:00,60.0000,180.0000,60.0000,60.

## Calculations

In [6]:
scoping_effort= np_data[:,np_data.shape[1]-4]/3600 #convert to hrs
approval_effort= np_data[:,np_data.shape[1]-3]/3600 #convert to hrs
intake_kit_effort= np_data[:,np_data.shape[1]-2]/3600 #convert to hrs
planning_effort= np_data[:,np_data.shape[1]-1]/3600 #convert to hrs

np_data=subtract_times(np_data,3,2) #total time from proj start to intake phase start (wait time)
proj_start_to_intake_start=np_data[:,np_data.shape[1]-1]

np_data=subtract_times(np_data,4,3) #total time from intake phase start to intake task start (wait time)
intake_phase_start_to_intake_task=np_data[:,np_data.shape[1]-1]
np_data=subtract_times(np_data,5,4) #total time from intake task start to intake phase end
intake_task_to_intake_phase_ends=np_data[:,np_data.shape[1]-1]

np_data=subtract_times(np_data,7,6) #total time from scoping phase start to scoping_task_assigned (wait time)
scoping_phase_starts_to_scoping_task=np_data[:,np_data.shape[1]-1]
np_data=subtract_times(np_data,8,7) #total time from scoping_task_assigned to scoping_phase_ends
scoping_task_to_scoping_phase_ends=np_data[:,np_data.shape[1]-1]

np_data=subtract_times(np_data,10,9) #total time from approval_phase_starts to approval_task_starts (wait time)
approval_phase_starts_to_approval_task=np_data[:,np_data.shape[1]-1]
np_data=subtract_times(np_data,11,10) #total time from approval_task_starts to first_quote
approval_task_to_1st_quote=np_data[:,np_data.shape[1]-1]

np_data=subtract_times(np_data,13,12) #total time from confirmintakekit_phase_starts to confirmintakekit_task_assigned (wait time)
CI_phase_start_to_task=np_data[:,np_data.shape[1]-1]
np_data=subtract_times(np_data,14,13) #total time from confirmintakekit_task_assigned to confirmintakekit_phase_end 
CI_task_to_CI_phase_end=np_data[:,np_data.shape[1]-1]

np_data=subtract_times(np_data,16,15) #total time from planning_phase_begin to planning_task_start (wait time)
Planning_phase_start_to_planning_task=np_data[:,np_data.shape[1]-1]
np_data=subtract_times(np_data,18,16) #total time from planning_task_start to 1st offer 
Planning_task_to_1st_offer=np_data[:,np_data.shape[1]-1]
np_data=subtract_times(np_data,18,17) #total time from 1st offer to planning_phase_end 
first_offer_to_planning_phase_end=np_data[:,np_data.shape[1]-1]

np_data=subtract_times(np_data,5,3) #intake effort 
intake_effort=np_data[:,np_data.shape[1]-1]

('shape before: ', (1763, 23))
('shape after: ', (1763, 24))
('shape before: ', (1763, 24))
('shape after: ', (1763, 25))
('shape before: ', (1763, 25))
('shape after: ', (1763, 26))
('shape before: ', (1763, 26))
('shape after: ', (1763, 27))
('shape before: ', (1763, 27))
('shape after: ', (1763, 28))
('shape before: ', (1763, 28))
('shape after: ', (1763, 29))
('shape before: ', (1763, 29))
('shape after: ', (1763, 30))
('shape before: ', (1763, 30))
('shape after: ', (1763, 31))
('shape before: ', (1763, 31))
('shape after: ', (1763, 32))
('shape before: ', (1763, 32))
('shape after: ', (1763, 33))
('shape before: ', (1763, 33))
('shape after: ', (1763, 34))
('shape before: ', (1763, 34))
('shape after: ', (1763, 35))
('shape before: ', (1763, 35))
('shape after: ', (1763, 36))


# Query for Falcon db

In [7]:
#all 
#query=\
'''
select top {}
   p."OrderID" as ProjectID, p."ProjectNumber",p."Category", t."TaskID", ph."Hours",
   t."SourceLanguage",t."TargetLanguage", p."PM",p."PmID", c."CustomerContactID", c."CompanyName"
from ProjectsActiveHours ph
   inner join Projects p
       on p."OrderID" = ph."OrderID"
   inner join ProjectsTasks t
       on t."TaskID"=ph."TaskID"
   inner join Customers c
       on c."CustomerContactID" = p."CustomerID"
where t."TaskID" > 0
    and ph."UserDepartment"='PM'
    and ph."Hours" > 0
    and ph."CustCompId" IN {}
    and ph."WorkDate">{}
order by p."OrderID" desc
'''.format(lim,fi,dat)

#with db_tools.connect_to_db('falcon_production').connect() as conn:
#    falcon_data = pd.read_sql(query, conn,)

#just hours per project
query=\
'''
select top {}
   p."OrderID" as ProjectID, sum(ph."Hours") as Hours
from ProjectsActiveHours ph
   inner join Projects p
       on p."OrderID" = ph."OrderID"
where ph."UserDepartment"='PM'
    and ph."Hours" > 0
    and ph."CustCompId" IN {}
    and ph."WorkDate">{}
group by p."OrderID"
order by p."OrderID" desc
'''.format(lim,fi,dat)

with db_tools.connect_to_db('falcon_production').connect() as conn:
    falcon_data = pd.read_sql(query, conn,)

falcon_data

#Convert hours to np array
falcon_data_hours=falcon_data['Hours'].values

## Removing possible outliers

In [8]:
proj_start_to_intake_start=rem_outliers (proj_start_to_intake_start)
intake_phase_start_to_intake_task=rem_outliers(intake_phase_start_to_intake_task)
intake_task_to_intake_phase_ends=rem_outliers(intake_task_to_intake_phase_ends)
scoping_phase_starts_to_scoping_task=rem_outliers(scoping_phase_starts_to_scoping_task)
scoping_task_to_scoping_phase_ends=rem_outliers(scoping_task_to_scoping_phase_ends)
approval_phase_starts_to_approval_task=rem_outliers(approval_phase_starts_to_approval_task)
approval_task_to_1st_quote=rem_outliers(approval_task_to_1st_quote)
CI_phase_start_to_task=rem_outliers(CI_phase_start_to_task)
CI_task_to_CI_phase_end=rem_outliers(CI_task_to_CI_phase_end)
Planning_phase_start_to_planning_task=rem_outliers(Planning_phase_start_to_planning_task)
Planning_task_to_1st_offer=rem_outliers(Planning_task_to_1st_offer)
first_offer_to_planning_phase_end=rem_outliers(first_offer_to_planning_phase_end)
intake_effort=rem_outliers(intake_effort)
scoping_effort=rem_outliers(scoping_effort)
approval_effort=rem_outliers(approval_effort)
intake_kit_effort=rem_outliers(intake_kit_effort)
falcon_data_hours=rem_outliers(falcon_data_hours)

## Timeline

In [9]:
#set columns
#means
cols_falcon_effort_mean= ['falcon_effort_mean']
cols_pantheon_effort_mean=['intake_effort_mean','scoping_effort_mean','approval_effort_mean','confirm_intake_effort_mean','planning_effort_mean']
cols_pantheon_project_mean=['project_start','proj_start_to_intake_start_mean','intake_phase_start_to_intake_task_mean','intake_task_to_intake_phase_ends_mean',
                       'scoping_phase_starts_to_scoping_task_mean','scoping_task_to_scoping_phase_ends_mean','approval_phase_starts_to_approval_task_mean',
                      'approval_task_to_1st_quote_mean','CI_phase_start_to_task_mean','CI_task_to_CI_phase_end_mean','Planning_phase_start_to_planning_task_mean',
                      'Planning_task_to_1st_offer_mean','first_offer_to_planning_phase_end_mean']
#stdevs
cols_falcon_effort_stdev= ['falcon_effort_stdev']
cols_pantheon_effort_stdev=['intake_effort_stdev','scoping_effort_stdev','approval_effort_stdev','confirm_intake_effort_stdev','planning_effort_stdev']
cols_pantheon_project_stdev=['project__start','proj_start_to_intake_start_stdev','intake_phase_start_to_intake_task_stdev','intake_task_to_intake_phase_ends_stdev',
                       'scoping_phase_starts_to_scoping_task_stdev','scoping_task_to_scoping_phase_ends_stdev','approval_phase_starts_to_approval_task_stdev',
                      'approval_task_to_1st_quote_stdev','CI_phase_start_to_task_stdev','CI_task_to_CI_phase_end_stdev','Planning_phase_start_to_planning_task_stdev',
                      'Planning_task_to_1st_offer_stdev','first_offer_to_planning_phase_end_stdev']


#set variables
#means
falcon_effort_mean=falcon_data_hours[0].mean()
intake_effort_mean=intake_effort[0].mean()
scoping_effort_mean=scoping_effort[0].mean()
approval_effort_mean=approval_effort[0].mean()
confirm_intake_effort_mean=intake_kit_effort[0].mean()
planning_effort_mean=planning_effort.mean()
project_start=0.01 #arbitrary small number (so it will show on the graph)
proj_start_to_intake_start_mean=proj_start_to_intake_start[0].mean()
intake_phase_start_to_intake_task_mean=intake_phase_start_to_intake_task[0].mean()
intake_task_to_intake_phase_ends_mean=intake_task_to_intake_phase_ends[0].mean()
scoping_phase_starts_to_scoping_task_mean=scoping_phase_starts_to_scoping_task[0].mean()
scoping_task_to_scoping_phase_ends_mean=scoping_task_to_scoping_phase_ends[0].mean()
approval_phase_starts_to_approval_task_mean=approval_phase_starts_to_approval_task[0].mean()
approval_task_to_1st_quote_mean=approval_task_to_1st_quote[0].mean()
CI_phase_start_to_task_mean=CI_phase_start_to_task[0].mean()
CI_task_to_CI_phase_end_mean=CI_task_to_CI_phase_end[0].mean()
Planning_phase_start_to_planning_task_mean=Planning_phase_start_to_planning_task[0].mean()
Planning_task_to_1st_offer_mean=Planning_task_to_1st_offer[0].mean()
first_offer_to_planning_phase_end_mean=first_offer_to_planning_phase_end[0].mean()

#stdev
falcon_effort_stdev=falcon_data_hours[0].std()
intake_effort_stdev=intake_effort[0].std()
scoping_effort_stdev=scoping_effort[0].std()
approval_effort_stdev=approval_effort[0].std()
confirm_intake_effort_stdev=intake_kit_effort[0].std()
planning_effort_stdev=planning_effort.std()
project__start=0.01
proj_start_to_intake_start_stdev=proj_start_to_intake_start[0].std()
intake_phase_start_to_intake_task_stdev=intake_phase_start_to_intake_task[0].std()
intake_task_to_intake_phase_ends_stdev=intake_task_to_intake_phase_ends[0].std()
scoping_phase_starts_to_scoping_task_stdev=scoping_phase_starts_to_scoping_task[0].std()
scoping_task_to_scoping_phase_ends_stdev=scoping_task_to_scoping_phase_ends[0].std()
approval_phase_starts_to_approval_task_stdev=approval_phase_starts_to_approval_task[0].std()
approval_task_to_1st_quote_stdev=approval_task_to_1st_quote[0].std()
CI_phase_start_to_task_stdev=CI_phase_start_to_task[0].std()
CI_task_to_CI_phase_end_stdev=CI_task_to_CI_phase_end[0].std()
Planning_phase_start_to_planning_task_stdev=Planning_phase_start_to_planning_task[0].std()
Planning_task_to_1st_offer_stdev=Planning_task_to_1st_offer[0].std()
first_offer_to_planning_phase_end_stdev=first_offer_to_planning_phase_end[0].std()

#make dictionaries 
#means
falcon_dict_mean={} 
for name in cols_falcon_effort_mean: #save values 
    falcon_dict_mean[name]=eval(name)     

pantheon_project_dict_mean={} 
for name in cols_pantheon_project_mean: #save values 
    pantheon_project_dict_mean[name]=eval(name)     

pantheon_effort_dict_mean={} 
for name in cols_pantheon_effort_mean: #save values 
    pantheon_effort_dict_mean[name]=eval(name)     

#stdev
falcon_dict_stdev={} 
for name in cols_falcon_effort_stdev: #save values 
    falcon_dict_stdev[name]=eval(name)     

pantheon_project_dict_stdev={} 
for name in cols_pantheon_project_stdev: #save values 
    pantheon_project_dict_stdev[name]=eval(name)     

pantheon_effort_dict_stdev={} 
for name in cols_pantheon_effort_stdev: #save values 
    pantheon_effort_dict_stdev[name]=eval(name)     


#set bars
d={} #empty dictionary to hold parameters
colors=[ #colors for the bars
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf',   # blue-teal
    'cornsilk','teal','dimgrey', 'dodgerblue', 'firebrick']

#falcon effort (mean)
f1=go.Bar(y=[''], x=[falcon_dict_mean[cols_falcon_effort_mean[0]]],name=cols_falcon_effort_mean[0],orientation = 'h',text=cols_falcon_effort_mean[0],textposition='auto',marker=dict(color='rgb(158,202,225)'))
d[str(f1)]=f1
#pantheon project (mean)
p_count=0
for i in range (len(cols_pantheon_project_mean)):
    strin='''go.Bar(y=[''], x=[pantheon_project_dict_mean[cols_pantheon_project_mean[{}]]],name=cols_pantheon_project_mean[{}],orientation = 'h',text=cols_pantheon_project_mean[{}],textposition='auto',marker=dict(color={}[{}]))'''.format(i,i,i,colors,i)
    count='p{}'.format(i)
    value=eval(strin) #example: p1= go.Bar(y=... cols_pantheon_effort_mean[1]
    d[str(count)]=value
    p_count+=1

#pantheon effort (mean)
b_count=0
for i in range (len(cols_pantheon_effort_mean)):
    strin='''go.Bar(y=[''], x=[pantheon_effort_dict_mean[cols_pantheon_effort_mean[{}]]],name=cols_pantheon_effort_mean[{}],orientation = 'h',text=cols_pantheon_effort_mean[{}],textposition='auto',marker=dict(color={}[{}]))'''.format(i,i,i,colors,i)
    count='b{}'.format(i)
    value=eval(strin) #example: b1= go.Bar(y=... cols_pantheon_effort_mean[1]
    d[str(count)]=value
    b_count+=1

#falcon effort (stdev)
ff1=go.Bar(y=[''], x=[falcon_dict_stdev['falcon_effort_stdev']],name=cols_falcon_effort_stdev[0],orientation = 'h',text=cols_falcon_effort_stdev[0],textposition='auto',marker=dict(color='rgb(158,202,225)'))
d[str(ff1)]=ff1
#pantheon project (stdev)
pp_count=0
for i in range (len(cols_pantheon_project_stdev)):
    strin='''go.Bar(y=[''], x=[pantheon_project_dict_stdev[cols_pantheon_project_stdev[{}]]],name=cols_pantheon_project_stdev[{}],orientation = 'h',text=cols_pantheon_project_stdev[{}],textposition='auto',marker=dict(color={}[{}]))'''.format(i,i,i,colors,i)
    count='pp{}'.format(i)
    value=eval(strin) 
    d[str(count)]=value
    pp_count+=1


#pantheon effort (stdev)
bb_count=0
for i in range (len(cols_pantheon_effort_stdev)):
    strin='''go.Bar(y=[''], x=[pantheon_effort_dict_stdev[cols_pantheon_effort_stdev[{}]]],name=cols_pantheon_effort_stdev[{}],orientation = 'h',text=cols_pantheon_effort_stdev[{}],textposition='auto',marker=dict(color={}[{}]))'''.format(i,i,i,colors,i)
    count='bb{}'.format(i)
    value=eval(strin) #example: b1= go.Bar(y=... cols_pantheon_effort_mean[1]
    d[str(count)]=value
    bb_count+=1
    
#subplots
fig=tools.make_subplots(rows=3, cols=2, specs=[[{},{}], [{},{}], [{},{}]],
                          shared_xaxes=False, shared_yaxes=False, vertical_spacing=0.12,
                          subplot_titles=('Pantheon Project Average Times (means)','Pantheon Project Average Times (stdev)','Falcon Effort Per Project Average Times (means)', 'Falcon Effort Per Project Average Times (stdev)','Pantheon Effort Per Project Average Times (means)','Pantheon Effort Per Project Average Times (stdev)'))

#assign f1 to subplot 2
fig.append_trace(f1, 2, 1)

#assign p0-p_last to subplot 1
for i in range (p_count):
    count='p{}'.format(i)
    fig.append_trace(d[str(count)], 1, 1)


#assign b0-b_last to subplot 3
for i in range (b_count):
    count='b{}'.format(i)
    fig.append_trace(d[str(count)], 3, 1)

#assign ff1 to subplot 2,2
fig.append_trace(ff1, 2, 2)

#assign pp1-pp_last to subplot 1,2
for i in range (pp_count):
    count='pp{}'.format(i)
    fig.append_trace(d[str(count)], 1, 2)

#assign bb1-bb_last to subplot 3,2
for i in range (bb_count):
    count='bb{}'.format(i)
    fig.append_trace(d[str(count)], 3, 2)


#set figure
fig['layout']['xaxis1'].update(title='Hours')
fig['layout']['xaxis2'].update(title='Hours')
fig['layout']['xaxis3'].update(title='Hours')

fig['layout']['xaxis4'].update(title='Hours')
fig['layout']['xaxis5'].update(title='Hours')
fig['layout']['xaxis6'].update(title='Hours')

fig['layout'].update(height=2000, width=1300, title='Stacked Subplots with Shared X-Axes')

iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]

